# PART 2:Association Analysis

# 1. Apriori Algorithm to find frequent itemsets

In [17]:
import numpy as np
import copy
import pandas as pd
import itertools
import sys
import numpy as np
import itertools
import sys
from itertools import chain

In [18]:
def supportCalculation(count,totalItems):
    support = (count/totalItems)*100
    return support

In [19]:
def freqSetGenerate(item,count,frequentSet,itemsCountSet):
    frequentSet.append(item)
    itemsCountSet.append(count)
    return frequentSet, itemsCountSet    

In [20]:
#calculation of frequent itemset of length-1
def frequentItemSetLength1(data, minsupport, totalItems):
    frequentSet =[]
    itemsCountSet = []

    #Gene Count Up and Down
    for i in range (0,data.shape[1]-1):
        upCnt=0
        downCnt =0
        for j in range(0,data.shape[0]):
            if (data[j][i] == 'Up'):
                upCnt+=1
            elif(data[j][i] == 'Down'):
                downCnt+=1
        if (supportCalculation(upCnt,totalItems) >= minsupport):
            freqSetGenerate([labelUp[i]],upCnt,frequentSet,itemsCountSet)
        if (supportCalculation(downCnt,totalItems) >= minsupport):
            freqSetGenerate([labelDown[i]],downCnt,frequentSet,itemsCountSet)
    #verify disease is more than support
    diseases=set(data[0:data.shape[0], data.shape[1]-1])
    for d in diseases:
        diseaseCnt =0
        for j in range(0, data.shape[0]):
            if (d == data[j][data.shape[1]-1]):
                diseaseCnt += 1
        if (supportCalculation(diseaseCnt,totalItems) >= minsupport):
            freqSetGenerate([str('G101_'+d)],diseaseCnt,frequentSet,itemsCountSet)
    return frequentSet, itemsCountSet

In [21]:
#calculation of frequent itemset of length-2
def frequentItemSetLength2(itemset, data, minsupport, totalItems):
    value = []

    for i in range(len(itemset) - 1):
        a=itemset[i][0].split('_')
        for j in range(i + 1, len(itemset)):
            b=itemset[j][0].split('_')
            if a[0] != b[0]:
                value.append([itemset[i][0], itemset[j][0]])

    # Up and Down Gene count                   
    frequentSet = []
    itemsCountSet = []
   
    for result in value:
        res1 = result[0].split('_')
        res2 = result[1].split('_')
        count = 0
        for i in range(0, data.shape[0]):
            if (data[i][int(res1[0][1:]) -1] == str(res1[1]) and
                    data[i][int(res2[0][1:]) -1] == str(res2[1])):
                count += 1
        if (supportCalculation(count,totalItems) >= minsupport):
            freqSetGenerate(result,count,frequentSet,itemsCountSet)
    return frequentSet, itemsCountSet

In [22]:
#calculation of frequent itemset for length>2
def frequentItemSetAllLengths(itemset, data, minsupport, totalItems):
    frequentSet =[]
    itemsCountSet = []
    length = len(itemset[0])
    for i in range(0, len(itemset) - 1):
        for j in range(i + 1, len(itemset)):
            if itemset[i][0:length - 1] == itemset[j][0:length - 1] and itemset[i][length - 1] != itemset[j][length - 1]:
                value = itemset[i][0:length - 1] + [itemset[i][length - 1]] + [itemset[j][length - 1]]
                # Calculating the Count for the 'Up' and 'Down' Genes
                itemList=[]
                for item in value:
                    itemList.append(item.split('_'))
                count = 0
                for k in range(0, data.shape[0]):
                    flag = True
                    for element in itemList:
                        if data[k][int(element[0][1:])-1] != str(element[1]):
                            flag = False
                            break
                    if flag:
                        count += 1
                if (supportCalculation(count,totalItems) >= minsupport):
                    freqSetGenerate(value,count,frequentSet,itemsCountSet)
    return frequentSet, itemsCountSet

In [23]:
#Starting point of the script
# Read the input data from the file
fileData = pd.read_csv("associationruletestdata.txt", sep='\t', lineterminator='\n', header=None)

nrows=len(fileData)
ncols=len(fileData[0])-1

#given support array
support=[30,40,50,60,70]

data = np.zeros(shape=(nrows,ncols))
#converting filedata to matrix
data = np.array(fileData)

# Label attachment to data in file
labelUp = []
labelDown = []
for i in range(0, data.shape[1]):
    labelUp.append('G' + str(i + 1) + '_Up')
    labelDown.append('G' + str(i + 1) + '_Down')
    
for i in range (0, len(support)):
    count =1;
    totalItem = 0
    print("Support is set to ",support[i])

    # calculating itemset for Length 1
    itemset, countSet = frequentItemSetLength1(data, support[i], data.shape[0])
    print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset)))
    totalItem+=len(itemset)
    count += 1
    # calculating itemset for length 2
    itemset2, countSet = frequentItemSetLength2(itemset, data, support[i], data.shape[0])
    if len(itemset2) > 0:
        print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset2)))
        count+=1
        totalItem += len(itemset2)
    #calculating itemset for length>2
    while(len(itemset2) > 1):
        itemset2, countSet = frequentItemSetAllLengths(itemset2, data, support[i], data.shape[0])
        if len(itemset2) > 0:
            print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset2)))
            count+=1
            totalItem += len(itemset2)
    print("Total Number of all itemsets:",totalItem)


Support is set to  30
No.of.length 1 frequent Itemsets=196
No.of.length 2 frequent Itemsets=5340
No.of.length 3 frequent Itemsets=5287
No.of.length 4 frequent Itemsets=1518
No.of.length 5 frequent Itemsets=438
No.of.length 6 frequent Itemsets=88
No.of.length 7 frequent Itemsets=11
No.of.length 8 frequent Itemsets=1
Total Number of all itemsets: 12879
Support is set to  40
No.of.length 1 frequent Itemsets=167
No.of.length 2 frequent Itemsets=753
No.of.length 3 frequent Itemsets=149
No.of.length 4 frequent Itemsets=7
No.of.length 5 frequent Itemsets=1
Total Number of all itemsets: 1077
Support is set to  50
No.of.length 1 frequent Itemsets=109
No.of.length 2 frequent Itemsets=63
No.of.length 3 frequent Itemsets=2
Total Number of all itemsets: 174
Support is set to  60
No.of.length 1 frequent Itemsets=34
No.of.length 2 frequent Itemsets=2
Total Number of all itemsets: 36
Support is set to  70
No.of.length 1 frequent Itemsets=7
Total Number of all itemsets: 7


# 2.Generation of Association Rules

In [24]:
#calculate combinations for generating rules using the itemset
def combinationCalculation(target, data):
    for i in range(len(data)):
        new_target = copy.copy(target)
        new_data = copy.copy(data)
        new_target.append(data[i])
        new_data = data[i + 1:]
        newTarget.append(new_target)
        combinationCalculation(new_target,new_data)

In [25]:
#calculating confidence of rule and setting true or false if itemset is frequent or not.
def confidenceCalculation(body, item, confFactor):
    body_string = ','.join(body)
    item_string = ','.join(item)
    if float((int(res_dict[item_string])/int(res_dict[body_string]))*100) >= confFactor:
        return True
    else:
        return False

In [26]:
count = 1
sup = 50
conf = 70

feqItemListSupported=[]
countOfItemSupported=[]


 # calculating itemset for Length 1
itemset, countSet = frequentItemSetLength1(data, sup, data.shape[0])
print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset)))
count += 1
feqItemListSupported.append(itemset)
countOfItemSupported.append(countSet)
# calculating itemset for length 2
itemset2, countSet = frequentItemSetLength2(itemset, data, sup, data.shape[0])
if len(itemset2) > 0:
    print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset2)))
    count+=1
    feqItemListSupported.append(itemset2)
    countOfItemSupported.append(countSet)
while(len(itemset2) > 1):
    itemset2, countSet = frequentItemSetAllLengths(itemset2, data, sup, data.shape[0])
    if len(itemset2) > 0:
        print("No.of.length %d frequent Itemsets=%d" %(count,len(itemset2)))
        count+=1
        feqItemListSupported.append(itemset2)
        countOfItemSupported.append(countSet)

No.of.length 1 frequent Itemsets=109
No.of.length 2 frequent Itemsets=63
No.of.length 3 frequent Itemsets=2


In [29]:
res_dict = {}
for i in range (len(feqItemListSupported)):
    for j in range(len(feqItemListSupported[i])):
        s = ','.join(feqItemListSupported[i][j])
        res_dict.setdefault(s, 0)
        res_dict[s] = countOfItemSupported[i][j]

        
combination=[]
headList=[]
bodyList=[]
for i in range(1,len(feqItemListSupported)):
    for j in range(0, len(feqItemListSupported[i])):
        newTarget =[]
        combinationCalculation(combination, feqItemListSupported[i][j])
        head=[]
        body=[]
        for ele in newTarget:
            val = [item for item in feqItemListSupported[i][j] if item not in ele]
            if val:
                ret = confidenceCalculation(val, feqItemListSupported[i][j], conf)
                if ret:
                    body.append(ele)
                    head.append(val)
        headList.append(head)
        bodyList.append(body)

In [30]:
def diff(list1, list2):
    c = set(list1).union(set(list2))  # or c = set(list1) | set(list2)
    d = set(list1).intersection(set(list2))  # or d = set(list1) & set(list2)
    return list(c - d)

In [31]:
def removeNestings(l): 
    for i in l: 
        if type(i) == list: 
            removeNestings(i) 
        else: 
            out.append(i) 
    return out

In [32]:
RULES=pd.DataFrame(columns=['RULE','HEAD','BODY'])
ruleset=[]
myList=[]
countset=0
temp3 = []
temp4=[]
out = []
for i in range (0, len(bodyList)):
    for j in range(0, len(bodyList[i])):
        ruleset.append("{} => {}".format(bodyList[i][j],headList[i][j]))
        myList=(bodyList[i][j],headList[i][j])
        countset+=1
        for word in myList:
            auxiliaryList = []
            if word not in auxiliaryList:
                auxiliaryList.append(word)
        l1=removeNestings(headList[i][j])
        l2=removeNestings(bodyList[i][j])
        temp3=list(set(l1)-set(l2))
        temp4=list(l1+l2)
        RULES.loc[len(RULES)]=pd.Series({'RULE': str(temp4),'HEAD': str(temp3), 'BODY': str(bodyList[i][j])})
print(ruleset)
print("total number of rules: ",countset)
RULES.to_csv('RULES.csv', sep=',')

["['G1_Up'] => ['G10_Down']", "['G10_Down'] => ['G1_Up']", "['G1_Up'] => ['G38_Down']", "['G38_Down'] => ['G1_Up']", "['G1_Up'] => ['G54_Up']", "['G54_Up'] => ['G1_Up']", "['G59_Up'] => ['G1_Up']", "['G1_Up'] => ['G67_Up']", "['G67_Up'] => ['G1_Up']", "['G1_Up'] => ['G70_Down']", "['G70_Down'] => ['G1_Up']", "['G1_Up'] => ['G72_Up']", "['G72_Up'] => ['G1_Up']", "['G2_Down'] => ['G28_Down']", "['G28_Down'] => ['G2_Down']", "['G38_Down'] => ['G2_Down']", "['G6_Up'] => ['G13_Down']", "['G13_Down'] => ['G6_Up']", "['G6_Up'] => ['G28_Down']", "['G28_Down'] => ['G6_Up']", "['G6_Up'] => ['G32_Down']", "['G32_Down'] => ['G6_Up']", "['G38_Down'] => ['G6_Up']", "['G59_Up'] => ['G6_Up']", "['G8_Up'] => ['G88_Down']", "['G88_Down'] => ['G8_Up']", "['G10_Down'] => ['G28_Down']", "['G28_Down'] => ['G10_Down']", "['G10_Down'] => ['G38_Down']", "['G38_Down'] => ['G10_Down']", "['G10_Down'] => ['G47_Up']", "['G47_Up'] => ['G10_Down']", "['G59_Up'] => ['G10_Down']", "['G10_Down'] => ['G70_Down']", "['G7

In [33]:
def template_1(type1, type2, list):
    countConf = 0
    rule = []
    if (type1.upper() == 'RULE'):
        for i in range (0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                count=0
                for p in range (0, len(list)):
                    if list[p] in bodyList[i][j] or list[p] in headList[i][j]:
                        count+=1
                if (str(type2.upper()) == '1'):
                    if count == 1:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif (str(type2.upper()) == 'NONE'):
                    if count == 0:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif str(type2.upper()) == 'ANY':
                    if count >= 1:
                        countConf += 1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    elif (type1.upper() == 'BODY'):
        for i in range (0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                count=0
                for p in range (0, len(list)):
                    if list[p] in bodyList[i][j]:
                        count+=1
                if (str(type2.upper()) == '1'):
                    if count == 1:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif (str(type2.upper()) == 'NONE'):
                    if count == 0:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif str(type2.upper()) == 'ANY':
                    if count >= 1:
                        countConf += 1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    elif (type1.upper() == 'HEAD'):
        for i in range (0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                count=0
                for p in range (0, len(list)):
                    if list[p] in headList[i][j]:
                        count+=1
                if (str(type2.upper()) == '1'):
                    if count == 1:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif (str(type2.upper()) == 'NONE'):
                    if count == 0:
                        countConf+=1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
                elif str(type2.upper()) == 'ANY':
                    if count >= 1:
                        countConf += 1
                        rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    return rule, countConf

In [34]:
def template_2(type1, type2):
    countConf = 0
    rule = []
    if (type1.upper() == 'RULE'):
        for i in range(0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                if (len(bodyList[i][j]) + len(headList[i][j])) > int(type2)-1:
                    countConf += 1
                    rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    elif (type1.upper() == 'BODY'):
        for i in range(0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                if (len(bodyList[i][j])) > int(type2)-1:
                    countConf += 1
                    rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    elif (type1.upper() == 'HEAD'):
        for i in range(0, len(bodyList)):
            for j in range(0, len(bodyList[i])):
                if (len(headList[i][j])) > int(type2)-1:
                    countConf += 1
                    rule.append("{} => {}".format(headList[i][j], bodyList[i][j]))
    return rule, countConf



In [35]:
print("------------------TEMPLATE:1--------------------------")
print("------------------------------------------------------")
rule, confCount = template_1("RULE", "ANY", ["G59_Up"])
print("RULE Has ANY of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("RULE", "NONE", ["G59_Up"])
print("RULE Has NONE of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("RULE", "1", ["G59_Up", "G10_Down"])
print("RULE Has 1 of [G59_Up, G10_Down]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("HEAD", "ANY", ["G59_Up"])
print("HEAD Has ANY of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("HEAD", "NONE", ["G59_Up"])
print("HEAD Has NONE of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("--------------------------------------------------------")

rule, confCount = template_1("HEAD", "1", ["G59_Up", "G10_Down"])
print("HEAD Has 1 of [G59_Up, G10_Down]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("BODY", "ANY", ["G59_Up"])
print("BODY Has ANY of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("BODY", "NONE", ["G59_Up"])
print("BODY Has NONE of [G59_Up]")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_1("BODY", "1", ["G59_Up", "G10_Down"])
print("BODY Has 1 of [G59_Up, G10_Down]")
print(rule)
print("The Count of the rules are ", confCount)


------------------TEMPLATE:1--------------------------
------------------------------------------------------
RULE Has ANY of [G59_Up]
["['G1_Up'] => ['G59_Up']", "['G6_Up'] => ['G59_Up']", "['G10_Down'] => ['G59_Up']", "['G59_Up'] => ['G13_Down']", "['G13_Down'] => ['G59_Up']", "['G28_Down'] => ['G59_Up']", "['G32_Down'] => ['G59_Up']", "['G38_Down'] => ['G59_Up']", "['G72_Up'] => ['G59_Up']", "['G59_Up'] => ['G72_Up']", "['G82_Down'] => ['G59_Up']", "['G59_Up'] => ['G82_Down']", "['G87_Up'] => ['G59_Up']", "['G88_Down'] => ['G59_Up']", "['G59_Up'] => ['G88_Down']", "['G96_Down'] => ['G59_Up']", "['G59_Up'] => ['G96_Down']", "['G72_Up', 'G82_Down'] => ['G59_Up']", "['G82_Down'] => ['G59_Up', 'G72_Up']", "['G72_Up'] => ['G59_Up', 'G82_Down']", "['G59_Up', 'G82_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G82_Down']", "['G72_Up', 'G96_Down'] => ['G59_Up']", "['G96_Down'] => ['G59_Up', 'G72_Up']", "['G59_Up', 'G96_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G96_Down']"]
The Co

In [36]:
print("-------------------TEMPLATE:2---------------------------")
rule, confCount = template_2("RULE", "3")
print("RULE, 3")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_2("HEAD", "2")
print("HEAD, 2")
print(rule)
print("The Count of the rules are ", confCount)
print("-------------------------------------------------------")

rule, confCount = template_2("BODY", "1")
print("BODY, 1")
print(rule)
print("The Count of the rules are ", confCount)

-------------------TEMPLATE:2---------------------------
RULE, 3
["['G72_Up', 'G82_Down'] => ['G59_Up']", "['G82_Down'] => ['G59_Up', 'G72_Up']", "['G72_Up'] => ['G59_Up', 'G82_Down']", "['G59_Up', 'G82_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G82_Down']", "['G72_Up', 'G96_Down'] => ['G59_Up']", "['G96_Down'] => ['G59_Up', 'G72_Up']", "['G59_Up', 'G96_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G96_Down']"]
The Count of the rules are  9
-------------------------------------------------------
HEAD, 2
["['G72_Up', 'G82_Down'] => ['G59_Up']", "['G59_Up', 'G82_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G82_Down']", "['G72_Up', 'G96_Down'] => ['G59_Up']", "['G59_Up', 'G96_Down'] => ['G72_Up']", "['G59_Up', 'G72_Up'] => ['G96_Down']"]
The Count of the rules are  6
-------------------------------------------------------
BODY, 1
["['G10_Down'] => ['G1_Up']", "['G1_Up'] => ['G10_Down']", "['G38_Down'] => ['G1_Up']", "['G1_Up'] => ['G38_Down']", "['G54_Up'] => ['G1_Up']", "

In [56]:
print("------------------TEMPLATE:3--------------------------")
print("------------------------------------------------------")
rule1, confCount1 = template_1("HEAD", "ANY", ["G10_Down"])
rule2, confCount2 = template_1("BODY", "1", ["G59_Up"])
rule3 = set(rule1).union(set(rule2))
print("1or1 HEAD has ANY of [G10_Down] or BODY has 1 of [G59_Up]")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

rule1, confCount1 = template_1("HEAD", "ANY", ["G10_Down"])
rule2, confCount2 = template_1("BODY", "1", ["G59_Up"])
rule3 = set(rule1).intersection(set(rule2))
print("1and1 HEAD has ANY of [G10_Down] and BODY has 1 of [G59_Up]")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

rule1, confCount1 = template_1("HEAD", "ANY", ["G10_Down"])
rule2, confCount2 = template_2("BODY", "2")
rule3 = set(rule1).union(set(rule2))
print("1or2 HEAD has ANY of [G10_Down] or BODY 2")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

rule1, confCount1 = template_1("HEAD", "ANY", ["G10_Down"])
rule2, confCount2 = template_2("BODY", "2")
rule3 = set(rule1).intersection(set(rule2))
print("1and2 HEAD has ANY of [G10_Down] and BODY 2")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

rule1, confCount1 = template_2("HEAD", "1")
rule2, confCount2 = template_2("BODY", "2")
rule3 = set(rule1).union(set(rule2))
print("2or2 HEAD 1 or BODY 2")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

rule1, confCount1 = template_2("HEAD", "1")
rule2, confCount2 = template_2("BODY", "2")
rule3 = set(rule1).intersection(set(rule2))
print("2and2 HEAD 1 and BODY 2")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

------------------TEMPLATE:3--------------------------
------------------------------------------------------
1or1 HEAD has ANY of [G10_Down] or BODY has 1 of [G59_Up]
{"['G10_Down', ['G47_Up']] => ['G47_Up']", "['G10_Down', ['G38_Down']] => ['G38_Down']", "['G28_Down', ['G59_Up']] => ['G59_Up']", "['G72_Up', 'G96_Down', ['G59_Up']] => ['G59_Up']", "['G10_Down', ['G1_Up']] => ['G1_Up']", "['G87_Up', ['G59_Up']] => ['G59_Up']", "['G88_Down', ['G59_Up']] => ['G59_Up']", "['G13_Down', ['G59_Up']] => ['G59_Up']", "['G10_Down', ['G88_Down']] => ['G88_Down']", "['G1_Up', ['G59_Up']] => ['G59_Up']", "['G10_Down', ['G28_Down']] => ['G28_Down']", "['G82_Down', ['G59_Up', 'G72_Up']] => ['G59_Up', 'G72_Up']", "['G72_Up', ['G59_Up']] => ['G59_Up']", "['G96_Down', ['G59_Up', 'G72_Up']] => ['G59_Up', 'G72_Up']", "['G10_Down', ['G94_Up']] => ['G94_Up']", "['G6_Up', ['G59_Up']] => ['G59_Up']", "['G96_Down', ['G59_Up']] => ['G59_Up']", "['G10_Down', ['G70_Down']] => ['G70_Down']", "['G10_Down', ['G59_U

# DEMO QUERIES

In [37]:
count = 1
sup = 60
conf = 65

feqItemListSupported=[]
countOfItemSupported=[]


 # calculating itemset for Length 1
itemset, countSet = size1Set(data, sup, data.shape[0])
print("Length=", count)
print("No.of.frequent Itemsets=",len(itemset))
count += 1
feqItemListSupported.append(itemset)
countOfItemSupported.append(countSet)
# calculating itemset for length 2
itemset2, countSet = size2Set(itemset, data, sup, data.shape[0])
if len(itemset2) > 0:
    print("Length=", count)
    print("No.of.frequent Itemsets=",len(itemset2))
    count+=1
    feqItemListSupported.append(itemset2)
    countOfItemSupported.append(countSet)
while(len(itemset2) > 1):
    itemset2, countSet = allSets(itemset2, data, sup, data.shape[0])
    if len(itemset2) > 0:
        print("Length=", count)
        print("No.of.frequent Itemsets=",len(itemset2))
        count+=1
        feqItemListSupported.append(itemset2)
        countOfItemSupported.append(countSet)

Length= 1
No.of.frequent Itemsets= 34
Length= 2
No.of.frequent Itemsets= 2


In [39]:
res_dict = {}
for i in range (len(feqItemListSupported)):
    for j in range(len(feqItemListSupported[i])):
        s = ','.join(feqItemListSupported[i][j])
        res_dict.setdefault(s, 0)
        res_dict[s] = countOfItemSupported[i][j]

combination=[]
headList=[]
bodyList=[]
for i in range(1,len(feqItemListSupported)):
    for j in range(0, len(feqItemListSupported[i])):
        newTarget =[]
        combinationCalculation(combination, feqItemListSupported[i][j])
        head=[]
        body=[]
        for ele in newTarget:
            val = [item for item in feqItemListSupported[i][j] if item not in ele]
            if val:
                ret = confidenceCalculation(val, feqItemListSupported[i][j], conf)
                if ret:
                    body.append(ele)
                    head.append(val)
        headList.append(head)
        bodyList.append(body)
        

In [40]:
print("------------------DEMO QUERIES-TEMPLATE:3--------------------------")
print("------------------------------------------------------")

rule1, confCount1 = template_1("BODY", "ANY", ["G1_Down"])
rule2, confCount2 = template_2("HEAD", "2")
rule3 = set(rule1).union(set(rule2))
print("1or2 BODY has ANY of [G1_Down] or HEAD 2")
print(rule3)
print("The Count of the rules are ", len(rule3))
print("------------------------------------------------------")

------------------DEMO QUERIES-TEMPLATE:3--------------------------
------------------------------------------------------
1or2 BODY has ANY of [G1_Down] or HEAD 2
set()
The Count of the rules are  0
------------------------------------------------------
